In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, linalg
# !pip install catboost
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings(action='ignore')
import re
from sklearn.model_selection import train_test_split

#자체 전처리 함수
import feature_engineering as fe 

     |████████████████████████████████| 76.6 MB 92.6 MB/s eta 0:00:01
     |████████████████████████████████| 15.3 MB 57.8 MB/s eta 0:00:01
     |████████████████████████████████| 9.4 MB 33.2 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 965 kB 84.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 62.4 MB/s eta 0:00:01
     |████████████████████████████████| 295 kB 89.8 MB/s eta 0:00:01


In [3]:
seed = 42

In [5]:
path = '../data/'
books = pd.read_csv(f"{path}books.csv")
test = pd.read_csv(f"{path}test_ratings.csv")
users = pd.read_csv(f"{path}users.csv")
sub = pd.read_csv(f"{path}sample_submission.csv")
train = pd.read_csv(f"{path}train_ratings.csv")

In [13]:
def data_split(data):
    X_train, X_test, y_train, y_test = train_test_split(data.drop(['user_id', 'isbn', 'rating'], axis=1), data['rating'], 
                                                    test_size=0.2, shuffle=True, random_state=seed)
    return X_train, X_test, y_train, y_test

In [5]:
def catboost_train_predict(X_train, X_test, y_train, y_test):
    cbr = CatBoostRegressor()
    cbr.fit(X_train, y_train)
    preds = cbr.predict(X_test)
    return cbr, preds

In [6]:
def rmse(y_test, preds):
    return mean_squared_error(y_test, preds)**0.5

In [7]:
def adjust_predict(y):
    if y < 1.0:
        return 1.0
    elif y > 10.0:
        return 10.0
    return y

In [21]:
def catboost_predict(cbr, x_test):
    df = x_test.drop(['user_id', 'isbn', 'rating'], axis=1)
    preds = cbr.predict(df)
    preds = list(map(adjust_predict, preds))
    return preds

## 첫 번째 전처리 적용

In [6]:
users1, books1 = fe.first_feature_engineering(users, books)

In [7]:
users1.head()

,user_id,age,location_country
0,8,NaN,canada
1,11400,49.0,canada
2,11676,NaN,NaN
3,67544,30.0,canada
4,85526,36.0,canada


In [8]:
books1.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_high
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,Actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,Actresses
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,Perennial,http://images.amazon.com/images/P/0060973129.0...,en,1940 1949,"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg,others
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,Medical,"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg,Medical
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,Fiction,A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg,Fiction
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,History,"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg,History


In [9]:
users_df = pd.get_dummies(
    users1,
    columns=['location_country']
)
users_df.head()

,user_id,age,location_country_afghanistan,location_country_albania,location_country_algeria,location_country_andorra,location_country_angola,location_country_antarctica,location_country_antiguaandbarbuda,location_country_argentina,...,location_country_ukraine,location_country_united kingdom,location_country_unitedarabemirates,location_country_uruguay,location_country_usa,location_country_uzbekistan,location_country_venezuela,location_country_yugoslavia,location_country_zambia,location_country_zimbabwe
0,8,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11400,49.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11676,NaN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,67544,30.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,85526,36.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
books1.columns

Index(['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher',
       'img_url', 'language', 'category', 'summary', 'img_path',
       'category_high'],
      dtype='object')

In [11]:
books1 = books1[['isbn','category_high','publisher','language']]

In [12]:
books_df = pd.get_dummies(
  books1, 
  columns=['category_high', 'publisher', 'language']
)
books_df.head()

,isbn,category_high_AIDS Disease,category_high_Abduction,category_high_Aboriginal Australians,category_high_Abortion,category_high_Abused wives,category_high_Accelerated readers,category_high_Accidents,category_high_Achievement motivation,category_high_Acting,...,language_ms,language_nl,language_no,language_pt,language_ro,language_ru,language_th,language_vi,language_zh-CN,language_zh-TW
0,0002005018,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0060973129,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0374157065,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0399135782,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0425176428,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
data1 = train.merge(books_df, on='isbn', how='inner').merge(users_df, on='user_id', how='inner')

In [14]:
data1.head()

,user_id,isbn,rating,category_high_AIDS Disease,category_high_Abduction,category_high_Aboriginal Australians,category_high_Abortion,category_high_Abused wives,category_high_Accelerated readers,category_high_Accidents,...,location_country_ukraine,location_country_united kingdom,location_country_unitedarabemirates,location_country_uruguay,location_country_usa,location_country_uzbekistan,location_country_venezuela,location_country_yugoslavia,location_country_zambia,location_country_zimbabwe
0,8,0002005018,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,074322678X,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,0887841740,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,1552041778,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,1567407781,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_train, X_test, y_train, y_test = data_split(data1)
cbr1, preds1 = catboost_train_predict(X_train, X_test, y_train, y_test)
rmse(y_test, preds1)

Learning rate set to 0.097676
0:	learn: 2.4290296	total: 61.4ms	remaining: 1m 1s
1:	learn: 2.4258226	total: 74.3ms	remaining: 37.1s
2:	learn: 2.4234257	total: 85.7ms	remaining: 28.5s
3:	learn: 2.4213291	total: 97.2ms	remaining: 24.2s
4:	learn: 2.4193641	total: 109ms	remaining: 21.7s
5:	learn: 2.4177258	total: 121ms	remaining: 20s
6:	learn: 2.4164189	total: 132ms	remaining: 18.8s
7:	learn: 2.4153039	total: 144ms	remaining: 17.8s
8:	learn: 2.4142521	total: 156ms	remaining: 17.2s
9:	learn: 2.4132682	total: 168ms	remaining: 16.6s
10:	learn: 2.4122981	total: 179ms	remaining: 16.1s
11:	learn: 2.4115575	total: 191ms	remaining: 15.7s
12:	learn: 2.4108673	total: 202ms	remaining: 15.3s
13:	learn: 2.4102682	total: 213ms	remaining: 15s
14:	learn: 2.4092662	total: 224ms	remaining: 14.7s
15:	learn: 2.4084355	total: 236ms	remaining: 14.5s
16:	learn: 2.4079132	total: 247ms	remaining: 14.3s
17:	learn: 2.4073956	total: 258ms	remaining: 14.1s
18:	learn: 2.4070258	total: 272ms	remaining: 14s
19:	learn: 2.

2.375398771538563

## 두 번째 전처리 적용

In [18]:
users.head()

,user_id,location,age,location_city,location_state,location_country
0,8,"timmins, ontario, canada",NaN,timmins,ontario,canada
1,11400,"ottawa, ontario, canada",49.0,ottawa,ontario,canada
2,11676,"n/a, n/a, n/a",NaN,n/a,n/a,n/a
3,67544,"toronto, ontario, canada",30.0,toronto,ontario,canada
4,85526,"victoria, british columbia, canada",36.0,victoria,british columbia,canada


In [19]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_high
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,Collins,http://images.amazon.com/images/P/0002005018.0...,en,Actresses,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,Actresses
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,Perennial,http://images.amazon.com/images/P/0060973129.0...,en,1940 1949,"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg,others
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,Medical,"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg,Medical
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,Fiction,A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg,Fiction
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,History,"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg,History


In [20]:
train_df2, test_df2 = fe.second_feature_engineering(users, books, train, test)

In [21]:
train_df2.head()

,user_id,isbn,rating,age,location_country,category,publisher,language,book_author
0,8,0002005018,4,3,0,0,0,0,0
1,67544,0002005018,7,3,0,0,0,0,0
2,123629,0002005018,8,3,0,0,0,0,0
3,200273,0002005018,8,3,0,0,0,0,0
4,210926,0002005018,9,3,0,0,0,0,0


In [22]:
test_df2.head()

,user_id,isbn,rating,age,location_country,category,publisher,language,book_author
0,11676,0002005018,0,3,2,0,0,0,0
1,116866,0002005018,0,3,0,0,0,0,0
2,152827,0060973129,0,4,0,1,1,0,1
3,157969,0374157065,0,3,1,2,2,0,2
4,67958,0399135782,0,3,1,3,3,0,3


In [23]:
X_train2, X_test2, y_train2, y_test2 = data_split(train_df2)
cbr2, preds2 = catboost_train_predict(X_train2, X_test2, y_train2, y_test2)
rmse(y_test2, preds2)

Learning rate set to 0.097676
0:	learn: 2.4290296	total: 12.9ms	remaining: 12.9s
1:	learn: 2.4258226	total: 25.1ms	remaining: 12.5s
2:	learn: 2.4234257	total: 36.6ms	remaining: 12.2s
3:	learn: 2.4213291	total: 48.7ms	remaining: 12.1s
4:	learn: 2.4193641	total: 60.5ms	remaining: 12s
5:	learn: 2.4177258	total: 72.5ms	remaining: 12s
6:	learn: 2.4164189	total: 84ms	remaining: 11.9s
7:	learn: 2.4153039	total: 95.7ms	remaining: 11.9s
8:	learn: 2.4142521	total: 108ms	remaining: 11.9s
9:	learn: 2.4132682	total: 121ms	remaining: 11.9s
10:	learn: 2.4122981	total: 133ms	remaining: 11.9s
11:	learn: 2.4115575	total: 145ms	remaining: 11.9s
12:	learn: 2.4108673	total: 156ms	remaining: 11.8s
13:	learn: 2.4102682	total: 168ms	remaining: 11.8s
14:	learn: 2.4092662	total: 179ms	remaining: 11.8s
15:	learn: 2.4084355	total: 191ms	remaining: 11.7s
16:	learn: 2.4079132	total: 203ms	remaining: 11.7s
17:	learn: 2.4073956	total: 215ms	remaining: 11.7s
18:	learn: 2.4070258	total: 227ms	remaining: 11.7s
19:	learn

2.375398771538563

## 세 번째 전처리 적용

In [9]:
train_df3, test_df3 = fe.third_feature_engineering(users, books, train, test)

In [10]:
train_df3.head()

,user_id,isbn,rating,age,location_country,category,language,book_title,book_author,publisher
0,8,0002005018,4,3,0.0,0,0,0,0,0
1,67544,0002005018,7,3,0.0,0,0,0,0,0
2,123629,0002005018,8,3,0.0,0,0,0,0,0
3,200273,0002005018,8,3,0.0,0,0,0,0,0
4,210926,0002005018,9,3,0.0,0,0,0,0,0


In [11]:
test_df3.head()

,user_id,isbn,rating,age,location_country,category,language,book_title,book_author,publisher
0,11676,0002005018,0,3,2.0,0,0,0,0,0
1,116866,0002005018,0,3,0.0,0,0,0,0,0
2,152827,0060973129,0,4,0.0,1,0,1,1,1
3,157969,0374157065,0,3,1.0,2,0,2,2,2
4,67958,0399135782,0,3,1.0,3,0,3,3,3


In [14]:
X_train3, X_test3, y_train3, y_test3 = data_split(train_df3)
cbr3, preds3 = catboost_train_predict(X_train3, X_test3, y_train3, y_test3)
rmse(y_test3, preds3)

Learning rate set to 0.097676
0:	learn: 2.4300413	total: 65.4ms	remaining: 1m 5s
1:	learn: 2.4273343	total: 85ms	remaining: 42.4s
2:	learn: 2.4249663	total: 102ms	remaining: 33.7s
3:	learn: 2.4228764	total: 119ms	remaining: 29.6s
4:	learn: 2.4214336	total: 130ms	remaining: 25.9s
5:	learn: 2.4195808	total: 145ms	remaining: 24.1s
6:	learn: 2.4182661	total: 157ms	remaining: 22.3s
7:	learn: 2.4171187	total: 167ms	remaining: 20.7s
8:	learn: 2.4161140	total: 179ms	remaining: 19.8s
9:	learn: 2.4151611	total: 190ms	remaining: 18.8s
10:	learn: 2.4134403	total: 199ms	remaining: 17.9s
11:	learn: 2.4126464	total: 209ms	remaining: 17.2s
12:	learn: 2.4118917	total: 218ms	remaining: 16.6s
13:	learn: 2.4105815	total: 227ms	remaining: 16s
14:	learn: 2.4099151	total: 235ms	remaining: 15.5s
15:	learn: 2.4092927	total: 244ms	remaining: 15s
16:	learn: 2.4086572	total: 253ms	remaining: 14.6s
17:	learn: 2.4077259	total: 261ms	remaining: 14.2s
18:	learn: 2.4073507	total: 270ms	remaining: 13.9s
19:	learn: 2.40

2.36698312947468

In [23]:
test['rating'] = catboost_predict(cbr3, test_df3)

In [24]:
test.head()

,user_id,isbn,rating
0,11676,0002005018,8.074625
1,116866,0002005018,7.890987
2,152827,0060973129,7.959788
3,157969,0374157065,7.969307
4,67958,0399135782,7.563158


# 하이퍼파라미터 튜닝

In [39]:
from sklearn.model_selection import KFold, GridSearchCV, train_test_split

In [47]:
parameters = {'depth'         : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [30, 50, 100] 
             }

In [48]:
grid = GridSearchCV(estimator=cbr3, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train3, y_train3)

0:	learn: 2.4324372	total: 80.4ms	remaining: 2.33s
1:	learn: 2.4320753	total: 93.8ms	remaining: 1.31s
2:	learn: 2.4317356	total: 202ms	remaining: 1.82s
3:	learn: 2.4313951	total: 231ms	remaining: 1.5s
4:	learn: 2.4310783	total: 248ms	remaining: 1.24s
5:	learn: 2.4307688	total: 287ms	remaining: 1.15s
6:	learn: 2.4304461	total: 345ms	remaining: 1.14s
7:	learn: 2.4301514	total: 439ms	remaining: 1.21s
8:	learn: 2.4298323	total: 498ms	remaining: 1.16s
9:	learn: 2.4295330	total: 567ms	remaining: 1.13s
10:	learn: 2.4292166	total: 650ms	remaining: 1.12s
11:	learn: 2.4289415	total: 729ms	remaining: 1.09s
12:	learn: 2.4286494	total: 782ms	remaining: 1.02s
13:	learn: 2.4283665	total: 816ms	remaining: 932ms
14:	learn: 2.4280867	total: 846ms	remaining: 846ms
15:	learn: 2.4278392	total: 936ms	remaining: 819ms
16:	learn: 2.4275056	total: 978ms	remaining: 748ms
17:	learn: 2.4272502	total: 1.01s	remaining: 675ms
18:	learn: 2.4269853	total: 1.07s	remaining: 619ms
19:	learn: 2.4267381	total: 1.12s	remain

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f6c5d23ebe0>,
             n_jobs=-1,
             param_grid={'depth': [6, 8, 10], 'iterations': [30, 50, 100],
                         'learning_rate': [0.01, 0.05, 0.1]})

In [49]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.03953922319368397

 The best parameters across ALL searched params:
 {'depth': 10, 'iterations': 100, 'learning_rate': 0.1}


In [52]:
cbr3 = CatBoostRegressor(depth=10, iterations=100, learning_rate=0.1)

In [53]:
cbr3.fit(X_train_cat, y_train_cat)

0:	learn: 2.4287142	total: 19.4ms	remaining: 1.92s
1:	learn: 2.4251260	total: 37.8ms	remaining: 1.85s
2:	learn: 2.4212894	total: 55.7ms	remaining: 1.8s
3:	learn: 2.4181647	total: 72.5ms	remaining: 1.74s
4:	learn: 2.4154493	total: 91.8ms	remaining: 1.74s
5:	learn: 2.4128245	total: 110ms	remaining: 1.72s
6:	learn: 2.4106774	total: 128ms	remaining: 1.7s
7:	learn: 2.4088176	total: 147ms	remaining: 1.69s
8:	learn: 2.4073086	total: 165ms	remaining: 1.67s
9:	learn: 2.4051850	total: 182ms	remaining: 1.64s
10:	learn: 2.4038387	total: 200ms	remaining: 1.62s
11:	learn: 2.4026537	total: 217ms	remaining: 1.59s
12:	learn: 2.4017296	total: 236ms	remaining: 1.58s
13:	learn: 2.4006524	total: 254ms	remaining: 1.56s
14:	learn: 2.3997871	total: 271ms	remaining: 1.54s
15:	learn: 2.3989080	total: 292ms	remaining: 1.53s
16:	learn: 2.3981042	total: 312ms	remaining: 1.52s
17:	learn: 2.3974599	total: 335ms	remaining: 1.52s
18:	learn: 2.3961188	total: 362ms	remaining: 1.54s
19:	learn: 2.3953575	total: 380ms	rema

In [55]:
preds3 = cbr3.predict(X_test3)
mean_squared_error(y_test3, preds3)**0.5

2.36698312947468

## Optuna로 하이퍼파라미터 튜닝

In [15]:
# !pip install optuna
import catboost
import optuna

def objective(trial):
    X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(train_df3.drop(['user_id', 'isbn', 'rating'], axis=1), train_df3['rating'], test_size=0.2, shuffle=True, random_state=42)

    param = {
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.3),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "max_depth": trial.suggest_int("max_depth", 1, 15),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
    }
    

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_uniform("subsample", 0.1, 1)

    gbm = CatBoostRegressor(**param, iterations = 10000)

    gbm.fit(X_train_cat, y_train_cat, eval_set = [(X_test_cat, y_test_cat)], verbose = 0, early_stopping_rounds = 100)

    preds = gbm.predict(X_test_cat)
    rmse = mean_squared_error(y_test_cat, preds)
    
    return rmse

study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 200, show_progress_bar = True)

[I 2022-11-03 05:28:53,561] A new study created in memory with name: no-name-967f66a4-daa4-447c-bb4f-863bccd47fc2


[I 2022-11-03 05:29:39,268] Trial 0 finished with value: 5.755407154610716 and parameters: {'learning_rate': 0.016498881912215407, 'colsample_bylevel': 0.05757697431705599, 'max_depth': 10, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 5.755407154610716.
[I 2022-11-03 05:31:02,454] Trial 1 finished with value: 5.829538733279276 and parameters: {'learning_rate': 0.014449736131573625, 'colsample_bylevel': 0.031222574596290173, 'max_depth': 8, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 9.34450307980409}. Best is trial 0 with value: 5.755407154610716.
[I 2022-11-03 05:32:20,430] Trial 2 finished with value: 5.902614932319697 and parameters: {'learning_rate': 0.0016980576178840263, 'colsample_bylevel': 0.013510311023006863, 'max_depth': 3, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 5.531565655937191}. Best is trial 0 with value: 5.755407154610716.
[I 2022-11-03 05:33:10,406] Trial 3